In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# ------------------------------------------------------------------------------
from scipy.stats import shapiro, kstest

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")


In [2]:
df_prod = pd.read_csv("../files/handm.csv")
df_prod.head()

,Unnamed: 0,productId,productName,brandName,url,price,stockState,comingSoon,colorName,isOnline,colors,colorShades,newArrival,mainCatCode,details,materials
0,0,834333001,Derby Shoes,H&M,https://www2.hm.com/en_us/productpage.08343330...,49.99,Available,False,Black,True,272628,Charcoal,False,men_shoes_dressed,Derby shoes with open lacing at front. Canvas ...,COMPOSITION\nSole:Thermoplastic rubber 100%\nU...
1,1,1258600003,Pile-Lined Slippers,H&M,https://www2.hm.com/en_us/productpage.12586000...,24.99,Available,False,Black,True,272628,Charcoal,False,men_shoes_slippers,Slippers in napped fabric. Decorative seam at ...,COMPOSITION\nLining:Polyester 100%\nUpper:Poly...
2,2,1036812001,Derby Shoes,H&M,https://www2.hm.com/en_us/productpage.10368120...,56.99,Available,False,Black,True,272628,Charcoal,False,men_shoes_dressed,Derby shoes in soft faux suede with open lacin...,COMPOSITION\nLining and insole:Cotton 100%\nUp...
3,3,1112924002,Fleece-lined Slippers,H&M,https://www2.hm.com/en_us/productpage.11129240...,29.99,Available,False,Dark blue,True,393B47,Bright blue|Cobalt blue|Dark blue|Indigo|Navy ...,False,men_shoes_slippers,Slippers in woven fabric with a moccasin seam ...,COMPOSITION\nLining:Polyester 100%\nUpper:Poly...
4,4,1119476020,Sneakers,H&M,https://www2.hm.com/en_us/productpage.11194760...,29.99,Available,False,White,True,FFFFFF,NaN,False,men_shoes_sneakers,"Sneakers with a padded upper edge and tongue, ...",COMPOSITION\nLining:Polyester 100%\nUpper:Poly...


In [3]:
df_prod = df_prod.drop(columns=['Unnamed: 0'])

In [4]:
df_prod['materials'].sample(5)

2556    COMPOSITION\nCotton 100% , Material:\nCotton ,...
7797    COMPOSITION\nPolyester 50%, Cotton 50% , ADDIT...
7698    COMPOSITION\nShell:Rayon 83%, Polyamide 17%\nL...
8778    COMPOSITION\nShell:Polyamide 87%, Spandex 13%\...
7434    COMPOSITION\nPolyester 61%, Cotton 39% , ADDIT...
Name: materials, dtype: object

In [5]:
#Crear bucle para que cuando 'organic', 'recycled' lo marque

df_prod['recycled'] = df_prod['materials'].apply(
    lambda x: 'Yes' if 'recycled' in str(x).lower() else 'No')

In [6]:
df_prod['organic'] = df_prod['materials'].apply(
    lambda x: 'Yes' if 'organic' in str(x).lower() else 'No')

In [7]:
df_prod.shape

(9677, 17)

In [8]:
df_prod['recycled'].value_counts()

recycled
Yes    5869
No     3808
Name: count, dtype: int64

In [9]:
df_prod['organic'].value_counts()

organic
No     9522
Yes     155
Name: count, dtype: int64

In [10]:
df_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9677 entries, 0 to 9676
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   productId    9677 non-null   int64  
 1   productName  9677 non-null   object 
 2   brandName    9677 non-null   object 
 3   url          9677 non-null   object 
 4   price        9677 non-null   float64
 5   stockState   9677 non-null   object 
 6   comingSoon   9677 non-null   bool   
 7   colorName    9677 non-null   object 
 8   isOnline     9677 non-null   bool   
 9   colors       9677 non-null   object 
 10  colorShades  6511 non-null   object 
 11  newArrival   9677 non-null   bool   
 12  mainCatCode  9677 non-null   object 
 13  details      9676 non-null   object 
 14  materials    9518 non-null   object 
 15  recycled     9677 non-null   object 
 16  organic      9677 non-null   object 
dtypes: bool(3), float64(1), int64(1), object(12)
memory usage: 1.1+ MB


In [11]:
df_prod['colorName']

0                  Black
1                  Black
2                  Black
3              Dark blue
4                  White
              ...       
9672               Beige
9673    Light denim blue
9674         Powder pink
9675         Beige/black
9676               Brown
Name: colorName, Length: 9677, dtype: object

In [12]:
for columna in df_prod:
    print(f'La columna {columna} tiene estos valores únicos:')
    print(df_prod[columna].unique())
    print('----------------------------')

La columna productId tiene estos valores únicos:
[ 834333001 1258600003 1036812001 ... 1232588003 1208310002 1227825002]
----------------------------
La columna productName tiene estos valores únicos:
['Derby Shoes' 'Pile-Lined Slippers' 'Fleece-lined Slippers' ...
 'Underwire Lace Bra' 'Slim Straight Ultra High Jeans'
 '2-pack Padded Soft-cup Cotton Bras']
----------------------------
La columna brandName tiene estos valores únicos:
['H&M']
----------------------------
La columna url tiene estos valores únicos:
['https://www2.hm.com/en_us/productpage.0834333001.html'
 'https://www2.hm.com/en_us/productpage.1258600003.html'
 'https://www2.hm.com/en_us/productpage.1036812001.html' ...
 'https://www2.hm.com/en_us/productpage.1232588003.html'
 'https://www2.hm.com/en_us/productpage.1208310002.html'
 'https://www2.hm.com/en_us/productpage.1227825002.html']
----------------------------
La columna price tiene estos valores únicos:
[ 49.99  24.99  56.99  29.99  59.99  34.99  54.99  74.99 119.

In [13]:
#Quitamos isOnline porque solo tiene un valor único
df_prod = df_prod.drop(columns='isOnline')

In [14]:
df_prod.columns

Index(['productId', 'productName', 'brandName', 'url', 'price', 'stockState',
       'comingSoon', 'colorName', 'colors', 'colorShades', 'newArrival',
       'mainCatCode', 'details', 'materials', 'recycled', 'organic'],
      dtype='object')

In [15]:
df_prod["mainCatCode"].value_counts()

mainCatCode
ladies_cardigansjumpers_jumpers         361
men_shirts_casual                       250
ladies_dresses_shortdresses             170
ladies_tops_longsleeve                  164
ladies_tops_shortsleeve                 162
                                       ... 
sportswear_men_activity_hiking            1
beauty_brushestools_skincaretools         1
men_hoodiessweatshirts                    1
ladies_shoes                              1
sportswear_women_clothing_tops_tanks      1
Name: count, Length: 355, dtype: int64

In [16]:

# Separar la columna 'categoria' por el carácter "_" en columnas nuevas
df_split = df_prod["mainCatCode"].str.split("_", expand=True)

# Renombrar las nuevas columnas con nombres genéricos
df_split.columns = [f"nivel_{i+1}" for i in range(df_split.shape[1])]

# Unir al DataFrame original 
df_prod= pd.concat([df_prod, df_split], axis=1)


In [17]:
df_prod[['nivel_1', 'nivel_2', 'nivel_3', 'nivel_4', 'nivel_5']]

,nivel_1,nivel_2,nivel_3,nivel_4,nivel_5
0,men,shoes,dressed,None,None
1,men,shoes,slippers,None,None
2,men,shoes,dressed,None,None
3,men,shoes,slippers,None,None
4,men,shoes,sneakers,None,None
...,...,...,...,...,...
9672,ladies,jeans,straight,None,None
9673,ladies,jeans,bootcut,None,None
9674,ladies,lingerie,bras,padded,None
9675,ladies,lingerie,bras,multipack,None


In [18]:
df_prod['nivel_1'].value_counts()


nivel_1
ladies        6395
men           2236
sportswear     566
beauty         226
Ladies         181
MEN             55
home            13
kids             5
Name: count, dtype: int64

In [19]:
# eliminar los tres ultimos niveles porque no aportan info relevante en este caso
cols_to_drop = df_prod.columns[-3:] 
df_sin_ultimos_niveles = df_prod.drop(columns=cols_to_drop)


In [20]:
df_prod = df_sin_ultimos_niveles

In [21]:
lista = ['nivel_1', 'nivel_2']
for col in lista:
    df_prod[col] = df_prod[col].str.capitalize()

In [22]:
df_prod[lista].value_counts()

nivel_1     nivel_2            
Ladies      Dresses                743
            Tops                   687
            Cardigansjumpers       510
            Accessories            485
            Basics                 424
            Trousers               419
Men         Tshirtstanks           396
Ladies      Maternity              384
            Lingerie               383
Sportswear  Women                  333
Ladies      Swimwear               297
Men         Shirts                 289
Ladies      Jeans                  287
Men         Jacketscoats           282
Ladies      Jacketscoats           274
            Shirtsblouses          270
            Shoes                  268
Sportswear  Men                    233
Men         Hoodiessweatshirts     233
Ladies      Nightwear              215
Men         Trousers               212
Ladies      Skirts                 210
            Premium                190
Men         Cardigansjumpers       175
            Socks               

In [23]:
# Lista para renombrar las columnas
columnas = ['productId', 'productName', 'brandName', 'url', 'price', 'stockState',
       'comingSoon', 'colorName', 'colors', 'colorShades', 'newArrival',
       'mainCatCode', 'details', 'materials', 'recycled', 'organic',
       'costumerSegment', 'productType']

In [24]:
df_prod.columns = columnas

In [25]:
df_prod.columns

Index(['productId', 'productName', 'brandName', 'url', 'price', 'stockState',
       'comingSoon', 'colorName', 'colors', 'colorShades', 'newArrival',
       'mainCatCode', 'details', 'materials', 'recycled', 'organic',
       'costumerSegment', 'productType'],
      dtype='object')

In [27]:
df_prod.to_csv('../files/products_hm_clean.csv', index=False)